# Crowling Berita

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid # Meskipun tidak digunakan untuk ID dari web, bisa untuk ID unik jika diperlukan

In [2]:
# URL homepage CNN Indonesia
url = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"}

# Ambil halaman utama
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []

# Loop semua link artikel di homepage
for artikel in soup.select("article a"):
    link = artikel.get("href")
    judul = artikel.get_text(strip=True)

    # Skip jika link kosong atau bukan http
    if not link or not link.startswith("http"):
        continue

    try:
        # Ambil halaman detail berita
        resp_detail = requests.get(link, headers=headers)
        soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # Ambil ID Berita dari link
        id_berita = None
        try:
            # Contoh path: /ekonomi/20250910153012-92-1234567/nama-artikel
            # Kita ambil bagian sebelum nama artikel, yaitu '20250910153012-92-1234567'
            id_segment = link.split("/")[-2]
            # Kemudian kita pisah dengan '-' dan ambil bagian terakhirnya
            id_berita = id_segment.split("-")[-1]
        except (IndexError, AttributeError):
            # Jika struktur URL berbeda dan gagal, ID akan tetap None
            id_berita = None

        # Ambil kategori dari link (path pertama setelah domain)
        kategori = None
        try:
            path = link.replace("https://www.cnnindonesia.com/", "")
            kategori = path.split("/")[0] if path else None
        except Exception:
            kategori = None

        # Ambil isi berita
        paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
        isi = " ".join(paragraf)

        if isi:  # hanya simpan kalau ada isi berita
            data.append({
                "id_berita": id_berita, # <-- Kolom baru ditambahkan di sini
                "judul": judul,
                "kategori": kategori,
                "isi": isi,
                "link": link
            })
            
        # --- PERUBAHAN SELESAI DI SINI ---

    except Exception as e:
        print(f"Gagal ambil {link}: {e}")

# Simpan ke dataframe Pandas
df = pd.DataFrame(data)

# Simpan ke file CSV
df.to_csv("berita_cnn_dengan_id.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil di notebook
pd.set_option("display.max_colwidth", 100)  # biar isi tidak kepotong
print("Proses selesai. Berikut adalah contoh datanya:")
display(df)

Proses selesai. Berikut adalah contoh datanya:


,id_berita,judul,kategori,isi,link
0,1274827,01Tutut Soeharto Gugat Menkeu ke PTUNEkonomi,ekonomi,"Putri Mantan Presiden RI ke-2 Soeharto, Siti Hardiyanti Hastuti Rukmana aliasTutut Soehartomelay...",https://www.cnnindonesia.com/ekonomi/20250917191944-532-1274827/tutut-soeharto-gugat-menkeu-ke-ptun
1,1274780,02KPK Sebut Khalid Basalamah Bocorkan Materi PenyidikanNasional,nasional,Komisi Pemberantasan Korupsi (KPK) mengatakan pemilik PT Zahra Oto Mandiri (Uhud Tour)Khalid Zee...,https://www.cnnindonesia.com/nasional/20250917173001-12-1274780/kpk-sebut-khalid-basalamah-bocor...
2,1274836,03VIDEO: Serangan Israel ke Gaza Bunuh Hampir 65 Ribu Warga PalestinaInternasional,internasional,Jumlah korban jiwa di Palestina atas serangan Israel meningkat menjadi 64.964 sejak konflik anta...,https://www.cnnindonesia.com/internasional/20250917193857-124-1274836/video-serangan-israel-ke-g...
3,1274834,"04Jejak Bima Permana Putra, Pedemo Hilang yang Ditemukan di MalangNasional",nasional,Polisi menyebut keberadaanBima Permana Putra(29) di Jakarta saat gelombangdemonstrasiakhir Agust...,https://www.cnnindonesia.com/nasional/20250917192751-12-1274834/jejak-bima-permana-putra-pedemo-...
4,1274720,05VIDEO: Prabowo Resmi Tunjuk Erick Thohir Jadi MenporaOlahraga,olahraga,Erick Thohir resmi menjadi Menteri Pemuda dan Olahraga baru setelah dilantik di Istana Merdeka p...,https://www.cnnindonesia.com/olahraga/20250917153806-182-1274720/video-prabowo-resmi-tunjuk-eric...
5,1273706,"06Reshuffle Kabinet, Prabowo Lantik Djamari Chaniago Jadi Menko PolkamNasional",nasional,Presiden RIPrabowo Subiantomelantik Djamari Chaniago jadi Menteri Koordinator Politik dan Keaman...,https://www.cnnindonesia.com/nasional/20250915090002-32-1273706/reshuffle-kabinet-prabowo-lantik...
6,1274722,Erick Thohir Buka Suara Soal Rangkap Jabatan Menpora dan Ketua PSSI,olahraga,Erick Thohiryang resmi jadi Menteri Pemuda dan Olahraga (Menpora) menyampaikan komentarnya soal ...,https://www.cnnindonesia.com/olahraga/20250917154231-178-1274722/erick-thohir-buka-suara-soal-ra...
7,1274593,"Reshuffle, Angga Raka Dilantik Jadi Kepala Badan Komunikasi Pemerintah",nasional,Presiden RIPrabowo Subiantomelantik Angga Raka Prabowo sebagai Kepala Badan Komunikasi Pemerinta...,https://www.cnnindonesia.com/nasional/20250917122808-32-1274593/reshuffle-angga-raka-dilantik-ja...
8,1274711,"Ditunjuk Jadi Menpora, Erick Thohir Tak Lagi Jabat Menteri BUMN",ekonomi,Erick Thohirtak lagi menjabat Menteri Badan Urusan Milik Negara (BUMN) setelah dilantik sebagai ...,https://www.cnnindonesia.com/ekonomi/20250917152025-92-1274711/ditunjuk-jadi-menpora-erick-thohi...
9,1274706,"Reshuffle Kabinet, Prabowo Angkat Qodari Jadi Kepala KSP",nasional,PresidenPrabowo Subiantomengangkat Muhammad Qodari jadi Kepala Kantor Staf Kepresidenan (KSP) me...,https://www.cnnindonesia.com/nasional/20250917151001-20-1274706/reshuffle-kabinet-prabowo-angkat...
